In [1]:
import os
import sys
import time
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from segment_mapping import *
import matplotlib.pyplot as plt
from apperception.utils import fetch_camera_config, fetch_camera_trajectory

In [2]:
# test_config = fetch_camera_config('samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385108912404.jpg', database)
# test_config

In [3]:
# mapping = map_imgsegment_roadsegment(test_config)

In [4]:
# test_file_path = '/home/yongming/workspace/research/apperception/v1.0-mini/samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385108912404.jpg'
# visualization(test_file_path, test_config, mapping)

In [5]:
%cd ../Yolov5_StrongSORT_OSNet
import sample_frame_tracker
# full_img_detection = sample_frame_tracker.run(test_file_path, save_vid=True, detect_only=True)
# full_img_detection

/home/yongming/workspace/research/apperception_new_local/apperception/Yolov5_StrongSORT_OSNet


strong_sort/deep/reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
/home/yongming/.pyenv/versions/apperception_venv/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
YOLOv5 🚀 2022-8-20 Python-3.8.3 torch-1.12.1+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


In [6]:
def display_detection(test_file_path, full_img_detection):
    import cv2
    test_frame = cv2.imread(test_file_path)
    for obj_idx, detection in full_img_detection.items():
        obj_cls, bbox = detection
        if obj_cls == 'car':
            x,y,w,h = list(map(int,bbox))
            cv2.rectangle(test_frame,(x-w//2,y-h//2),(x+w//2,y+h//2),(0,255,0),2)
            cv2.putText(test_frame, '_'.join([obj_cls, str(obj_idx)]), (x+w//2+5,y+h//2+5),0,0.3,(0,255,0))
    cv2.imshow('detection', test_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
from detection_estimation.utils import *
import matplotlib.pyplot as plt

In [8]:
# ego car trajectory
def prepare_ego(test_video):
    ego_trajectory = get_ego_trajectory(test_video)
    video_trajectory = fetch_camera_trajectory(test_video, database)
    sorted_ego_configs = [fetch_camera_config(e['fileName'], database) for e in video_trajectory]
    return sorted_ego_configs, ego_trajectory

In [9]:
# ego_speed = get_ego_speed(ego_trajectory)

In [10]:
# [time_to_nearest_frame(test_video, point.timestamp) for point in ego_trajectory]

In [11]:
import datetime
# test_timestamp = datetime.datetime(2018, 8, 27, 8, 51, 32, 162404, tzinfo=datetime.timezone.utc)
# timestamp_to_nearest_trajectory(ego_trajectory, test_timestamp)

In [12]:
# test_point = [1772, 865, 0.0]
# point_to_nearest_trajectory(test_point, ego_trajectory)

In [13]:
from detection_estimation.sample_plan_algorithms import *

In [14]:
base_dir = '/home/yongming/workspace/research/apperception_new_local/boston-seaport'
test_img_base_dir = '/home/yongming/workspace/research/apperception/v1.0-mini/'

In [15]:
# import pickle
# with open(os.path.join(base_dir, f'frames.pickle'), "rb") as f:
#     df_sample_data = pickle.loads(f.read())
# df_sample_data
# i = 0
# for frame in df_sample_data['scene-0655-CAM_FRONT']['frames']:
#     if frame[2] == 1194:
#         print(i)
#         break
#     i += 1

In [16]:
from detection_estimation.detection_estimation import *
# ### Integration
# target_config_idx = 218
# video = 'scene-0655-CAM_FRONT'
# configs = df_sample_data[video]
# sorted_ego_config = [dict(zip(configs['columns'], frame))
#                      for frame in configs['frames']]
# len(sorted_ego_config)
# # all_car_loc3d = 

In [17]:
# current_ego_config = sorted_ego_config[target_config_idx]

In [18]:
car_loc3d_ground_truth = [(1991, 874), (1949.181, 873.164)]

In [19]:
from shapely.geometry import Point, Polygon
def generate_sample_plan_once(video, ego_config, mapping, next_frame_num, car_loc3d=None, target_car_detection=None, all_detection_info=None):
    if all_detection_info is None:
        assert target_car_detection and car_loc3d
        x,y,w,h = list(map(int, target_car_detection))
        car_loc2d = (x, y+h//2)
        car_bbox2d = (x-w//2,y-h//2,x+w//2,y+h//2)
        car_bbox3d = None
        all_detections = []
        all_detections.append(obj_detection('car_1', car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
        all_detection_info = construct_all_detection_info(
            current_frame, cam_segment_mapping, ego_trajectory, ego_config, all_detections)
    if all_detection_info:
        print(all_detection_info[0].road_type)
    next_sample_plan = generate_sample_plan(video, next_frame_num, all_detection_info,  50)
    next_frame = None
    if next_sample_plan.get_next_sample_frame_info():
        next_sample_frame_name, next_sample_frame_num, next_sample_frame_time = (
            next_sample_plan.get_next_sample_frame_info())
        print("next frame name", next_sample_frame_name)
        print("next frame num", next_sample_frame_num)
    #     print(next_sample_plan.action)
        next_frame = cv2.imread(test_img_base_dir+next_sample_frame_name)
#         cv2.imshow("next_frame", next_frame)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
    return next_sample_plan, next_frame


In [32]:
def construct_estimated_all_detection_info(current_frame, cam_segment_mapping, ego_config, ego_trajectory):
    all_detections = []
    full_img_detection = sample_frame_tracker.run(current_frame, save_vid=True, detect_only=True)
#     display_detection(current_frame, full_img_detection)
    for obj_idx, detection in full_img_detection.items():
        obj_cls, bbox = detection
        x,y,w,h = list(map(int,bbox))
        car_loc2d = (x,y+h//2)
#         print(car_loc2d)
        car_bbox2d = (x-w//2,y-h//2,x+w//2,y+h//2)
        car_bbox3d = None
        estimate_3d = detection_to_img_segment(car_loc2d, cam_segment_mapping)
        if estimate_3d and estimate_3d.road_segment_info.segment_type in ['lane', 'laneSection']:
            car_loc3d = tuple(Polygon(estimate_3d.road_segment_info.segment_polygon).centroid.coords)
#             print(tuple(car_loc3d))
            all_detections.append(obj_detection('car_1', car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
    print("all_detections", all_detections)
    all_detection_info = construct_all_detection_info(
        current_frame, cam_segment_mapping, ego_trajectory, ego_config, all_detections)
    return all_detection_info

In [33]:
def dry_run(sorted_ego_configs, start_frame_num, ego_trajectory, video):
    skipped_frame_num = []
    current_frame_num = start_frame_num
    action_type_counts = {}
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    display_video = cv2.VideoWriter(f'sampled_frames_{video.replace("/", "_")}.avi',fourcc, 10, (1600, 900))
    start_time = time.time()
    total_detection_time = 0
    total_sample_plan_time = 0
    for i in range(len(sorted_ego_configs)-1):
        current_ego_config = sorted_ego_configs[i]
        next_frame_num = sorted_ego_configs[i+1]['frameNum']
        if current_ego_config['frameNum'] != current_frame_num:
            skipped_frame_num.append(current_ego_config['frameNum'])
            continue
        cam_segment_mapping = map_imgsegment_roadsegment(current_ego_config)
        current_frame = test_img_base_dir + current_ego_config['fileName']
        display_video.write(cv2.imread(current_frame))
        start_detection_time = time.time()
        all_detection_info = construct_estimated_all_detection_info(
            current_frame, cam_segment_mapping, current_ego_config, ego_trajectory)
        total_detection_time += time.time()-start_detection_time
        start_generate_sample_plan = time.time()
        next_sample_plan, next_frame = generate_sample_plan_once(
            video, current_ego_config, cam_segment_mapping, next_frame_num, all_detection_info=all_detection_info)
        total_sample_plan_time += time.time() - start_generate_sample_plan
        next_action_type = next_sample_plan.get_action_type()
        if next_action_type not in action_type_counts:
            action_type_counts[next_action_type] = 1
        else:
            action_type_counts[next_action_type] += 1
        current_frame_num = next_sample_plan.get_next_frame_num(next_frame_num)
    display_video.release()
    print("sorted_ego_config_length", len(sorted_ego_configs))
    print("number of skipped", len(skipped_frame_num))
    print(skipped_frame_num)
    print(action_type_counts)
    total_run_time = time.time()-start_time
    num_runs = len(sorted_ego_configs) - len(skipped_frame_num)
    print("total_run_time", total_run_time)
    print("avg run time", total_run_time/num_runs)
    print("total_detection_time", total_detection_time)
    print("avg detection time", total_detection_time/num_runs)
    print("total_generate_sample_plan_time", total_sample_plan_time)
    print("avg generate_sample_plan time", total_sample_plan_time/num_runs)
    total_mapping_time = total_run_time-total_detection_time-total_sample_plan_time
    print("total_mapping_time", total_mapping_time)
    print("avg mapping time", total_mapping_time/num_runs)

In [34]:
test_video1 = 'CAM_FRONT/n008-2018-08-27'
sorted_ego_configs1, ego_trajectory1 = prepare_ego(test_video1)
dry_run(sorted_ego_configs1, 2, ego_trajectory1, test_video1)


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.3097507953643799
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections [obj_detection(id='car_1', car_loc3d=((1810.3982084033205, 860.0461031162465),), car_loc2d=(1058, 532), car_bbox3d=None, car_bbox2d=(970, 466, 1146, 532))]
really contained True
area 367.7220652969542
lane
ego_exit_segment_action action type: ego_exit_segment,
        start time: 2018-08-27 08:51:32.112404+00:00,
        finish time: 2018-08-27 08:51:32.912404+00:00,
        start loc: [1772.1438, 866.30286],
        end loc: [1778.386, 866.4177, 0.0]
        estimated time: 0:00:00.800000
next frame name sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385092362404.jpg
next frame num 41


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.25127482414245605
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections []


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


...

/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


total mapping time:  0.17557048797607422
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
all_detections []
total mapping time:  0.16657543182373047


/home/yongming/workspace/research/apperception_new_local/apperception/optimization_playground/detection_estimation/utils.py:45: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  a, b = line.boundary


Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000


KeyboardInterrupt: 

In [ ]:
test_video2 = 'CAM_FRONT/n008-2018-08-01'
sorted_ego_configs2, ego_trajectory2 = prepare_ego(test_video2)
dry_run(sorted_ego_configs2, 2, ego_trajectory2, test_video2)
